In [1]:
import cv2
from cv2 import VideoCapture
import numpy as np
from tqdm import tqdm
from typing import Callable

In [2]:
def calcBackground(
    videoPath: str,
    numSamples: int = 100,
    frameTransform: Callable[[np.ndarray], np.ndarray] = None,
) -> np.ndarray:
    # Open Video
    cap = VideoCapture(videoPath)

    # Get Video Length
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Randomly select up to 50 frames
    frameIds = length * np.random.uniform(size=min(length, numSamples))

    # Store selected frames in an array
    frames = []
    for fid in tqdm(frameIds, desc="reading frames"):
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()

        # Apply transform if needed
        if frameTransform is not None:
            frame = frameTransform(frame)

        # Convert to grayscale
        frames.append(frame)

    # Calculate the median along the time axis
    medianFrame = np.median(frames, axis=0).astype(dtype=np.uint8)

    return medianFrame

In [3]:
def toGrayscale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


bgGray = calcBackground("worms.avi", numSamples=100, frameTransform=toGrayscale)

reading frames: 100%|██████████| 50/50 [02:05<00:00,  2.51s/it]


In [4]:
def extractBoxes(
    image: np.ndarray,
    imgBackground: np.ndarray,
    diffThresh: int = 75,
    dimThresh: int = 40,
) -> np.ndarray:
    # Calculate difference between background and image
    diff = np.abs(image.astype(np.int32) - imgBackground.astype(np.int32)).astype(np.uint8)

    # Turn differences mask to black & white according to a threshold value
    _, mask = cv2.threshold(diff, diffThresh, 255, cv2.THRESH_BINARY)

    # find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bboxes = []
    for c in contours:
        rect = cv2.boundingRect(c)

        # Skip too small bounding boxes
        if rect[2] < dimThresh and rect[3] < dimThresh:
            continue

        bboxes.append(rect)

    return bboxes

In [5]:
def extractRois(image: np.ndarray, bboxes: list, width: int = 200, height: int = 200):
    rois = []
    for box in bboxes:
        x, y, w, h = box

        # Calculate center of bbox
        xCenter, yCenter = x + w // 2, y + h // 2

        # Get bottom left corner
        x0, y0 = xCenter - width // 2, yCenter - height // 2
        x0, y0 = max(0, x0), max(0, y0)

        roi = image[y0 : y0 + height, x0 : x0 + width]

        rois.append(roi)

    return rois

In [10]:
def FindRois(videoPath: str, diffThresh: int = 75, dimThresh: int = 40):
    cap = VideoCapture(videoPath)

    while True:
        ret, image = cap.read()
        if ret == False:
            break

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        bboxes = extractBoxes(gray, bgGray, diffThresh, dimThresh)

        rois = extractRois(image, bboxes)

        # Draw bboxes
        for box in bboxes:
            x, y, w, h = box

            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 3)

        cv2.imshow("BBOXES", image)

    cap.release()
    cv2.destroyAllWindows()


FindRois("worms.avi")

In [7]:
cv2.destroyAllWindows()